In [35]:
from osgeo import gdal

In [61]:
out_put_full_path:str=r'./data/max_02.png'
in_put_full_path:str=r'./data/max.tif'

In [38]:
# 此种方式可以转存为 png ，但只有单波段，且范围为 0-190左右，且部分内容有缺失
translate_options = gdal.TranslateOptions(format='JPEG',
                                          outputType=gdal.GDT_Byte,
                                          scaleParams=[''],
                                          # scaleParams=[min_val, max_val],
                                          )
# translate_options = gdal.TranslateOptions(rgbExpand='RGB', format='GTiff'
#                                           )

gdal.Translate(destName=out_put_full_path, srcDS=in_put_full_path, options=translate_options)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000018E9977E690> >

尝试使用cv2将 tif -> png  
直接转换不行

In [26]:
import os
from PIL import Image

In [29]:
full_path=r'./data/max.tif'
img=Image.open(full_path)

In [30]:
type(img)

PIL.TiffImagePlugin.TiffImageFile

In [28]:
img.show()

In [24]:
out_img=img.convert("RGB")

In [39]:
#out_img

In [17]:
out_full_path:str=r'./data/max_rgb.png'
out_img.save(out_full_path,quality=95)

#### 尝试使用 gdal 进行转换

In [ ]:
from osgeo import gdal

translate_options = gdal.TranslateOptions(format='JPEG',
                                          outputType=gdal.GDT_Byte,
                                          scaleParams=[''],
                                          # scaleParams=[min_val, max_val],
                                          )

gdal.Translate(destName='test.jpg', srcDS='test.tif', options=translate_options)

#### 
参考文献: https://gis.stackexchange.com/questions/245562/how-to-transform-a-geotiff-dem-to-png-tiles-encoding-elevation-as-rgb-colors  
此种方式输出的rgb图片有问题，放弃

In [42]:
import rasterio
import numpy as np

In [45]:
with rasterio.open('./data/max.tif') as src:
    dem = src.read(1)

r = np.zeros(dem.shape)
g = np.zeros(dem.shape)
b = np.zeros(dem.shape)

r += np.floor_divide((100000 + dem * 10), 65536)
g += np.floor_divide((100000 + dem * 10), 256) - r * 256
b += np.floor(100000 + dem * 10) - r * 65536 - g * 256



c:\Users\evase\Anaconda3\envs\new_oil_gdal\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in floor_divide
  
c:\Users\evase\Anaconda3\envs\new_oil_gdal\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in floor_divide
  if __name__ == '__main__':


In [48]:
meta = src.meta
# meta(dtype=rasterio.uint8,nodata=0,count=3)


In [52]:
meta['dtype']

'float32'

In [55]:
meta['dtype']=rasterio.uint8
meta['count']=3
meta['nodata']=0

In [56]:
with rasterio.open('./data/max_outfile.tif', 'w', **meta) as dst:
    dst.write_band(1, r.astype(rasterio.uint8))
    dst.write_band(2, g.astype(rasterio.uint8))
    dst.write_band(3, b.astype(rasterio.uint8))

#### 尝试将 geotiff -> geojson  
参考文章: https://gis.stackexchange.com/questions/368784/convert-geotiff-to-geojson  
https://rasterio.readthedocs.io/en/latest/  
但注意此种方式存储完并不包含`val`数组

In [60]:
import rasterio
import rasterio.features
import rasterio.warp
import io,json

with rasterio.open('./data/max.tif') as dataset:

    # Read the dataset's valid data mask as a ndarray.
    mask = dataset.dataset_mask()

    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(
            mask, transform=dataset.transform):

        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(
            dataset.crs, 'EPSG:4326', geom, precision=6)

        # Print GeoJSON shapes to stdout.
        # print(geom)
        with io.open('./data/max.json','w') as f:
            json.dump(geom, f, ensure_ascii=False)